In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from custom_losses import *
from custom_metrics import *

from models import build_unet, build_w_net
import os
import cv2

In [3]:
# Define paths to your dataset
image_folder = 'DRIVE/training/images'
label_folder = 'DRIVE/training/1st_manual'

# Get list of all files in the image and label directories
image_files = sorted(os.listdir(image_folder))
label_files = sorted(os.listdir(label_folder))



In [4]:
# Function to preprocess images and labels
def preprocess(image_path, label_path):
    # Convert the TensorFlow Tensor object to a string
    image_path = image_path.decode('utf-8')
    label_path = label_path.decode('utf-8')
    # Load image and label
    image = cv2.imread(image_path)
    label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
    # Resize images and labels
    image_resized = cv2.resize(image, (512, 512))
    label_resized = cv2.resize(label, (512, 512))
    
    # Normalize images and labels
    image_normalized = image_resized.astype(np.float32) / 255.0
    label_normalized = label_resized.astype(np.float32) / 255.0
    label_normalized = label_normalized[:, :, np.newaxis]
    return image_normalized, label_normalized



In [5]:
# Function to load the dataset into tf.data.Dataset
def load_dataset(image_folder, label_folder):
    # Create file paths
    image_paths = [os.path.join(image_folder, f) for f in image_files]
    label_paths = [os.path.join(label_folder, f) for f in label_files]
    # Create a TensorFlow dataset from the file paths
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
    
    # Map the preprocessing function to each element in the dataset
    dataset = dataset.map(lambda img_path, lbl_path: tf.numpy_function(
                          func=preprocess, inp=[img_path, lbl_path], 
                          Tout=[tf.float32, tf.float32]))
    
    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(4)  # Set batch size to 32, modify as needed
    
    return dataset



In [6]:
# Load the dataset
dataset = load_dataset(image_folder, label_folder)

In [7]:
for batch in dataset:
    image, label = batch
    for i in label:
        print(i.shape)

(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)


In [16]:
num_epoch = 4000
step = 50
# Define the cosine annealing learning rate scheduler
def cosine_annealing_schedule(epoch, lr):
    initial_lr = lr  # Use the current learning rate as the initial rate
    if lr%step == 0:
        lr = 0.5 * initial_lr * (1 + np.cos(np.pi * (epoch) / num_epoch))
    return lr

# Create the LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(cosine_annealing_schedule)

In [17]:
wnet = build_w_net((512, 512, 3))  # Input size is 128x128 grayscale image
wnet.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_40 (Conv2D)             (None, 512, 512, 8)  224         ['input_3[0][0]']                
                                                                                                  
 conv2d_41 (Conv2D)             (None, 512, 512, 8)  584         ['conv2d_40[0][0]']              
                                                                                                  
 max_pooling2d_8 (MaxPooling2D)  (None, 256, 256, 8)  0          ['conv2d_41[0][0]']        

In [14]:

wnet.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1E-2), loss={"out1":DiceLoss(), "out2":DiceLoss()}, metrics=[iou_metric])


In [15]:
wnet.fit(x=dataset, epochs=100, verbose=2, callbacks=[lr_scheduler])

Epoch 1/100
5/5 - 4s - loss: 1.4704 - out1_loss: 0.7357 - out2_loss: 0.7347 - out1_iou_metric: 0.0805 - out2_iou_metric: 0.0791 - lr: 0.0100 - 4s/epoch - 867ms/step
Epoch 2/100
5/5 - 2s - loss: 1.4698 - out1_loss: 0.7354 - out2_loss: 0.7344 - out1_iou_metric: 0.0804 - out2_iou_metric: 0.0791 - lr: 0.0100 - 2s/epoch - 331ms/step
Epoch 3/100
5/5 - 2s - loss: 1.4687 - out1_loss: 0.7349 - out2_loss: 0.7338 - out1_iou_metric: 0.0803 - out2_iou_metric: 0.0791 - lr: 0.0100 - 2s/epoch - 337ms/step
Epoch 4/100
5/5 - 2s - loss: 1.4685 - out1_loss: 0.7348 - out2_loss: 0.7337 - out1_iou_metric: 0.0802 - out2_iou_metric: 0.0790 - lr: 0.0099 - 2s/epoch - 337ms/step
Epoch 5/100
5/5 - 2s - loss: 1.4666 - out1_loss: 0.7339 - out2_loss: 0.7327 - out1_iou_metric: 0.0801 - out2_iou_metric: 0.0791 - lr: 0.0097 - 2s/epoch - 335ms/step
Epoch 6/100
5/5 - 2s - loss: 1.4660 - out1_loss: 0.7336 - out2_loss: 0.7324 - out1_iou_metric: 0.0800 - out2_iou_metric: 0.0790 - lr: 0.0095 - 2s/epoch - 342ms/step
Epoch 7/10


KeyboardInterrupt

